In [1]:
import re
import pandas as pd
import string
from stemming.porter2 import stem
import numpy as np


import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

import spacy
from collections import defaultdict


import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel



import pyLDAvis
import pyLDAvis.gensim  
import matplotlib.pyplot as plt


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
np.random.seed(101)

In [3]:


class TextExtractor:
    def __init__(self, path):
        self.path = path
        self.list_of_comedies = self.extract_comedy()

    def extract_comedy(self):  # extracts comedies from file
        start_keyword = "SCENE:"
        end_keyword = "FINIS"
        comedy = []
        comedies=[]

        flag = False

        with open(self.path, "r", encoding='utf-8') as f:#change
            for line in f:
                if re.search(start_keyword, line):
                    flag = True
                elif flag:
                    comedy.append(line.rstrip())
                    #if line.isupper() and  end_keyword not in line and '.' not in line:
                     #   titles.append(line.strip())
                    if re.search(end_keyword, line):
                        flag = False
                        comedies.append("\n".join(comedy))
                        comedy=[]
        return comedies
    



        

                        


In [4]:
comedies_1 = TextExtractor("Aristofanes_11_comedies.txt")
comedies_2 = TextExtractor("Aristofanes_vol2.txt")
comedies_1.extract_comedy()
comedies_2.extract_comedy()
total_comedies = comedies_1.list_of_comedies + comedies_2.list_of_comedies



In [5]:

class DataProcessor:
    def __init__(self,documents):
        self.documents= documents
        #self.titles = titles
    


    def clean_text(self):
            
        # Remove all-uppercase words
        self.documents = [re.sub(r'\b[A-Z]+\b', '', text) for text in self.documents]

        #remove numbers
        self.documents = [re.sub(r'\b[0-9]+\b', '', text) for text in self.documents]

        #remove hypen
        self.documents = [re.sub(r'\-+', " ", text)for text in self.documents]

        #remove small words
        #self.documents = [[token for token in text if len(token)> 4] for text in self.documents]
        

        # Remove punctuation
        self.documents = [text.translate(str.maketrans(' ', ' ', string.punctuation))  for text in self.documents]

        #lowercase
        self.documents= [text.lower() for text in self.documents]
        
        # Remove new line characters
        self.documents = [re.sub('\s+', ' ', sent) for sent in self.documents]
        
        self.documents = [text.split() for text in self.documents]
        
        #Remove stopwords
        stop_words = set(stopwords.words('english'))
        stop_words.update(['demostenes', "nicias", 'agoracritus', 'demos', 'dicaeopolis', 'herald', 'amphitheus', 'ambassadors', 
        'pseudartabas', 'theorus', 'euripides', ' cephisophon', 'lamachus','megarian', 'maidens', 'boeotian', 'nicarchus', 'husbandman', 'bridesmaid', 'informer', 'messengers', 
        'elderls', 'trygaeus', 'hermes', 'tumult', 'hierocles', 'cleonymus', 'lysistrata', 'calonicé', 'lampito', 'stratyllis', 'magistrate', 'cinesias', 'lacedaemonians',
        'polucharides', 'loungers', 'strepsiades', 'phidippides', 'servant', 'socrates', 'disciples', 'pasias', 'lender', 'amynias', 'chaerephon', 
        'philocleon', 'bdelycleon', 'sosias', 'xanthias', 'baker', 'wife', 'accuser', 'euelpides', 'pisthetaerus', 'epops', 'trochilus', 'phoenicopterus', 
        'heralds', 'priest', 'poet', 'prophet', 'meton', 'geometrician', 'commissioner', 'dealer', 'iris', 'parricide', 'cinesias', 'informer', 'prometheus', 
        'posidon', 'triballus', 'heracles','birds','dionysus', 'charon', 'aeacus', 'persephone', 'attendant', 'inkeeper', 'wive', 'euripides', 'aeschylus', 'pluto'
        'frogs', 'mnesilochus', 'agathon', 'clisthenes', 'prytanis', 'scythian','paphlagonian', 'myrrhiné',
        'praxagora', 'blepyrus', 'citizens','chremylus', 'cario', 'plutus', 'blepsidemus', 'poverty', 'informer', 'sycophant', 'priest', 'rustics', 
        'philocleon', 'bdelycleon', 'sosias', 'xanthias', 'bacer', 'accuser', 'euelpides', 'pisthetaerus', 'epops', 'trochilus', 
        'phoenicopterus', 'poet', 'prophet', 'meton', 'commissioner', 'iris', 'parricide', 'prometheus', 'posidon', 'triballus', 'heracles', 'pisthetaerus, messenger'])
        self.documents = [[word for word in text if word not in stop_words] for text in self.documents]
        


        return self.documents
    


    def lemmatize(self): #this has to change!!!!!!!! Do I need it?
        rules = spacy.load('en_core_web_sm') #needs downloading beforehand
        lemmatized_data = []
        for comedy in self.documents: 
            lemmas= [rules(word)[0].lemma_ for word in comedy]
            lemmatized_data.append(lemmas)
        self.documents = lemmatized_data


    def tokenize(self):
        self.documents = [[stem(word) for word in comedy] for comedy in self.documents]
        
        
        
    def create_dictionary(self):
        corpus = dict(zip(self.titles, self.documents))
        return corpus

<>:30: DeprecationWarning: invalid escape sequence '\s'
<>:30: DeprecationWarning: invalid escape sequence '\s'
C:\Users\user\AppData\Local\Temp\ipykernel_9816\4112835500.py:30: DeprecationWarning: invalid escape sequence '\s'
  self.documents = [re.sub('\s+', ' ', sent) for sent in self.documents]


In [6]:
data = DataProcessor(total_comedies)
data.clean_text()
data.lemmatize()
data.tokenize()
all_comedies = data.documents
print(len(total_comedies))

11


In [7]:
word_frequency = gensim.corpora.Dictionary(all_comedies)
print(word_frequency)


Dictionary<6452 unique tokens: ['abl', 'abomin', 'abrad', 'abreast', 'absenc']...>


In [8]:
word_frequency.filter_extremes(no_below=1, no_above=0.5, keep_n=4000)

In [9]:
bow= [word_frequency.doc2bow(comedy) for comedy in all_comedies]

In [10]:
print(bow)

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 2), (8, 1), (9, 2), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 2), (16, 1), (17, 2), (18, 1), (19, 1), (20, 1), (21, 1), (22, 6), (23, 2), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 3), (31, 2), (32, 1), (33, 1), (34, 2), (35, 1), (36, 2), (37, 1), (38, 2), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1), (50, 1), (51, 2), (52, 2), (53, 1), (54, 1), (55, 1), (56, 1), (57, 4), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 2), (65, 1), (66, 3), (67, 3), (68, 2), (69, 1), (70, 1), (71, 1), (72, 1), (73, 1), (74, 1), (75, 4), (76, 1), (77, 2), (78, 1), (79, 2), (80, 1), (81, 1), (82, 1), (83, 1), (84, 1), (85, 1), (86, 1), (87, 1), (88, 1), (89, 1), (90, 1), (91, 1), (92, 1), (93, 1), (94, 6), (95, 2), (96, 1), (97, 2), (98, 1), (99, 1), (100, 3), (101, 1), (102, 1), (103, 1), (104, 3), (105, 1), (106, 1), (107, 1), (108, 3), (109, 1), (110, 1)

In [26]:
My_lda= gensim.models.LdaMulticore(bow,num_topics=3, id2word= word_frequency, passes=10)

In [34]:

for idx, topic in My_lda.print_topics(-1):
    print(f"Topic: {idx} Word composition:{topic}\n")

Topic: 0 Word composition:0.005*"moon" + 0.005*"trough" + 0.004*"debt" + 0.004*"creditor" + 0.003*"pigeon" + 0.003*"studi" + 0.003*"deposit" + 0.003*"femal" + 0.003*"lesson" + 0.003*"masculin"

Topic: 1 Word composition:0.007*"coax" + 0.005*"bottl" + 0.003*"dine" + 0.003*"assault" + 0.003*"baggag" + 0.002*"contest" + 0.002*"sport" + 0.002*"pest" + 0.002*"handsom" + 0.002*"recov"

Topic: 2 Word composition:0.004*"sausag" + 0.004*"seller" + 0.003*"town" + 0.003*"beak" + 0.003*"pylo" + 0.003*"lawsuit" + 0.003*"tribun" + 0.002*"read" + 0.002*"genius" + 0.002*"feather"

Topic: 3 Word composition:0.007*"heav" + 0.004*"wee" + 0.004*"beetl" + 0.003*"plume" + 0.003*"conclud" + 0.003*"helmet" + 0.002*"megara" + 0.002*"gorgon" + 0.002*"beggar" + 0.002*"thrush"



In [35]:

coherence_model_lda = CoherenceModel(model=My_lda, texts=all_comedies, dictionary=word_frequency, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.5195829834424407


In [24]:
for doc_idx, doc in enumerate(all_comedies):
    doc_bow = My_lda.id2word.doc2bow(doc)  # Convert the document to a bag-of-words representation
    doc_topics = My_lda.get_document_topics(doc_bow)  # Get the topics for the document

    print(f"Document {doc_idx} Topics:")
    for topic, prob in doc_topics:
        print(f"  Topic {topic}: Probability {prob}")
    print("\n")


Document 0 Topics:
  Topic 0: Probability 0.9994799494743347


Document 1 Topics:
  Topic 0: Probability 0.9993970990180969


Document 2 Topics:
  Topic 2: Probability 0.9993912577629089


Document 3 Topics:
  Topic 2: Probability 0.9994547963142395


Document 4 Topics:
  Topic 1: Probability 0.9994959831237793


Document 5 Topics:
  Topic 0: Probability 0.9995205998420715


Document 6 Topics:
  Topic 1: Probability 0.9662846922874451
  Topic 2: Probability 0.03340006247162819


Document 7 Topics:
  Topic 2: Probability 0.999333381652832


Document 8 Topics:
  Topic 1: Probability 0.999030590057373


Document 9 Topics:
  Topic 1: Probability 0.9989624619483948


Document 10 Topics:
  Topic 0: Probability 0.9988452196121216




#Using only libraries

In [42]:
import re
import pandas as pd
import string
from stemming.porter2 import stem
import numpy as np


import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

import spacy
from collections import defaultdict


import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel



import pyLDAvis
import pyLDAvis.gensim  
import matplotlib.pyplot as plt

import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer


nltk.download('stopwords')
nltk.download('wordnet')  # Add this line to download WordNet


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...


True

In [43]:


class TextExtractor:
    def __init__(self, path):
        self.path = path
        self.list_of_comedies = self.extract_comedy()

    def extract_comedy(self):  # extracts comedies from file
        start_keyword = "SCENE:"
        end_keyword = "FINIS"
        comedy = []
        comedies=[]

        flag = False

        with open(self.path, "r", encoding='utf-8') as f:#change
            for line in f:
                if re.search(start_keyword, line):
                    flag = True
                elif flag:
                    comedy.append(line.rstrip())
                    #if line.isupper() and  end_keyword not in line and '.' not in line:
                     #   titles.append(line.strip())
                    if re.search(end_keyword, line):
                        flag = False
                        comedies.append("\n".join(comedy))
                        comedy=[]
        return comedies
    



        

                        


In [44]:
np.random.seed(101)

In [45]:
comedies_1 = TextExtractor("Aristofanes_11_comedies.txt")
comedies_2 = TextExtractor("Aristofanes_vol2.txt")
comedies_1.extract_comedy()
comedies_2.extract_comedy()
total_comedies = comedies_1.list_of_comedies + comedies_2.list_of_comedies



In [46]:
print(total_comedies)

['\n       *       *       *       *       *\n\nTHE KNIGHTS\n\n\nDEMOSTHENES. Oh! alas! alas! Oh! woe! oh! woe! Miserable Paphlagonian![5]\nmay the gods destroy both him and his cursed advice! Since that evil day\nwhen this new slave entered the house he has never ceased belabouring us\nwith blows.\n\nNICIAS. May the plague seize him, the arch-fiend--him and his lying\ntales!\n\nDEMOSTHENES. Hah! my poor fellow, what is your condition?\n\nNICIAS. Very wretched, just like your own.\n\nDEMOSTHENES. Then come, let us sing a duet of groans in the style of\nOlympus.[6]\n\nDEMOSTHENES AND NICIAS. Boo, hoo! boo, hoo! boo, hoo! boo, hoo! boo, hoo!\nboo, hoo!!\n\nDEMOSTHENES. Bah! \'tis lost labour to weep! Enough of groaning! Let us\nconsider how to save our pelts.\n\nNICIAS. But how to do it! Can you suggest anything?\n\nDEMOSTHENES. Nay! you begin. I cede you the honour.\n\nNICIAS. By Apollo! no, not I. Come, have courage! Speak, and then I will\nsay what I think.\n\nDEMOSTHENES. "Ah! would 

In [47]:
lm = WordNetLemmatizer()
stemmer = SnowballStemmer("english")


def lem_stem(text):
    return stemmer.stem(lm.lemmatize(text, pos='v'))


def tokenize_lemmatize(text):
    return [lem_stem(token) for token in gensim.utils.simple_preprocess(text)
            if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3]


for comedy in total_comedies:
    lem_stem(comedy)
    tokenize_lemmatize(comedy)

In [49]:
print(tokenize_lemmatize(total_comedies[0]))

['knight', 'demosthen', 'ala', 'ala', 'miser', 'paphlagonian', 'god', 'destroy', 'curs', 'advic', 'evil', 'slave', 'enter', 'hous', 'ceas', 'belabour', 'blow', 'nicia', 'plagu', 'seiz', 'arch', 'fiend', 'lie', 'tale', 'demosthen', 'poor', 'fellow', 'condit', 'nicia', 'wretch', 'like', 'demosthen', 'come', 'sing', 'duet', 'groan', 'style', 'olympus', 'demosthen', 'nicia', 'demosthen', 'lose', 'labour', 'weep', 'groan', 'consid', 'save', 'pelt', 'nicia', 'suggest', 'demosthen', 'begin', 'cede', 'honour', 'nicia', 'apollo', 'come', 'courag', 'speak', 'think', 'demosthen', 'tell', 'tell', 'nicia', 'dare', 'express', 'thought', 'pomp', 'euripid', 'demosthen', 'prithe', 'spare', 'pelt', 'veget', 'leav', 'master', 'nicia', 'bolt', 'like', 'breath', 'demosthen', 'follow', 'bolt', 'nicia', 'bolt', 'speed', 'demosthen', 'speed', 'nicia', 'splendid', 'masturb', 'slowli', 'bolt', 'quick', 'firm', 'speed', 'demosthen', 'bolt', 'bolt', 'speed', 'nicia', 'demosthen', 'faith', 'fear', 'omen', 'bode', 

In [52]:
word_frequency = gensim.corpora.Dictionary(all_comedies)
print(word_frequency)


Dictionary<6679 unique tokens: ['abl', 'abomin', 'abrad', 'abreast', 'absenc']...>


In [53]:
word_frequency.filter_extremes(no_below=2, no_above=0.5, keep_n=2000)

In [54]:
bow= [word_frequency.doc2bow(comedy) for comedy in all_comedies]

In [58]:
print(bow)


[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 2), (6, 1), (7, 2), (8, 1), (9, 1), (10, 1), (11, 2), (12, 2), (13, 1), (14, 1), (15, 1), (16, 6), (17, 2), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 3), (25, 1), (26, 2), (27, 1), (28, 2), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 2), (41, 2), (42, 1), (43, 1), (44, 1), (45, 1), (46, 4), (47, 1), (48, 1), (49, 1), (50, 1), (51, 2), (52, 1), (53, 3), (54, 3), (55, 2), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 4), (62, 1), (63, 2), (64, 2), (65, 1), (66, 1), (67, 1), (68, 1), (69, 1), (70, 1), (71, 1), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 2), (78, 1), (79, 2), (80, 1), (81, 3), (82, 1), (83, 1), (84, 1), (85, 1), (86, 1), (87, 3), (88, 1), (89, 1), (90, 3), (91, 1), (92, 1), (93, 1), (94, 1), (95, 2), (96, 2), (97, 1), (98, 2), (99, 1), (100, 1), (101, 1), (102, 1), (103, 1), (104, 1), (105, 1), (106, 1), (107, 1), (108, 1), (109, 1), (110, 1)

In [80]:
My_lda= gensim.models.LdaMulticore(bow,num_topics=5, id2word= word_frequency, passes=10)


In [78]:
for idx, topic in My_lda.print_topics(-1):
    print(f"Topic: {idx} Word composition:{topic}\n")

Topic: 0 Word composition:0.005*"bottl" + 0.005*"town" + 0.005*"sting" + 0.004*"ass" + 0.004*"beak" + 0.003*"tribun" + 0.003*"contest" + 0.003*"baggag" + 0.003*"read" + 0.003*"urn"

Topic: 1 Word composition:0.020*"heav" + 0.010*"beetl" + 0.006*"hymen" + 0.006*"hymenaeus" + 0.006*"maker" + 0.006*"libat" + 0.005*"husbandmen" + 0.004*"crest" + 0.004*"town" + 0.004*"pestl"

Topic: 2 Word composition:0.007*"paphlagonian" + 0.006*"sausag" + 0.006*"seller" + 0.004*"pylo" + 0.004*"dine" + 0.003*"moon" + 0.003*"lawsuit" + 0.003*"trough" + 0.003*"pest" + 0.002*"sail"

Topic: 3 Word composition:0.004*"conclud" + 0.004*"beggar" + 0.004*"plume" + 0.004*"bearer" + 0.003*"bowl" + 0.003*"recov" + 0.003*"acharnian" + 0.003*"aid" + 0.003*"stir" + 0.003*"gorgon"



In [82]:

coherence_model_lda = CoherenceModel(model=My_lda, texts=all_comedies, dictionary=word_frequency, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.5099646217837733
